In [1]:
import pandas as pd
import numpy as np

In [2]:
col=['HPO1', 'HPO2', 'rel_type'] 
# pos_link = pd.read_csv('/home/ukjung18/HiG2Vec/data/HPO.tsv', names=col, sep='\t')
pos_link = pd.read_csv('/home/ukjung18/HiG2Vec/data/HPO_full.tsv', names=col, sep='\t')

In [3]:
hpo_uniq = pos_link['HPO1'].unique()
len(hpo_uniq)

31122

In [4]:
hpo = list(pd.unique(pos_link['HPO1']))
hpo2 = list(pd.unique(pos_link['HPO2']))
for i in hpo2:
    if i not in hpo:
        # print('hpo: {}'.format(i))
        hpo.append(i)
print(len(hpo))

31161


In [5]:
import scipy.sparse as sp
import torch
from collections import defaultdict

In [6]:
node_dict = defaultdict(int) # {HP:0000001 : 1}
for i in range(len(hpo)):
    node_dict[hpo[i]] = i

In [7]:
# Split edge set for training and testing
rel_len = len(pos_link['HPO1'])
u = []
v = []
for i in range(rel_len):
    u.append(node_dict[pos_link['HPO1'][i]])
    v.append(node_dict[pos_link['HPO2'][i]])

u = torch.IntTensor(u)
v = torch.IntTensor(v)

eids = np.arange(len(u))
eids = np.random.permutation(eids)
test_size = int(len(eids) * 0.1)
train_size = len(u) - test_size
test_pos_u, test_pos_v = np.array(u[eids[:test_size]]), np.array(v[eids[:test_size]])
train_pos_u, train_pos_v = np.array(u[eids[test_size:]]), np.array(v[eids[test_size:]])

# Find all negative edges and split them for training and testing
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())), shape=(len(hpo), len(hpo)))
adj_neg = 1 - adj.todense() - np.eye(len(hpo))
neg_u, neg_v = np.where(adj_neg != 0)

neg_eids = np.random.choice(len(neg_u), len(u))
test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]
train_neg_u, train_neg_v = neg_u[neg_eids[test_size:]], neg_v[neg_eids[test_size:]]

In [8]:
key_list = list(node_dict.keys())
val_list = list(node_dict.values())

In [9]:
train_neg_x = []
train_neg_y = []

for i in range(len(train_neg_u)):
    position_x = val_list.index(train_neg_u[i])
    train_neg_x.append(key_list[position_x])
    position_y = val_list.index(train_neg_v[i])
    train_neg_y.append(key_list[position_y])

zero = [0]*len(train_neg_u)
train_neg = pd.DataFrame(list(zip(train_neg_x, train_neg_y, zero)))

In [10]:
train_pos_x = []
train_pos_y = []

train_pos_u = list(train_pos_u)
train_pos_v = list(train_pos_v)

for i in range(len(train_pos_u)):
    position_x = val_list.index(train_pos_u[i])
    train_pos_x.append(key_list[position_x])
    position_y = val_list.index(train_pos_v[i])
    train_pos_y.append(key_list[position_y])
  
ones = [1]*len(train_pos_u)
train_pos = pd.DataFrame(list(zip(train_pos_x, train_pos_y, ones)))

In [11]:
train_set = pd.concat([train_pos, train_neg], ignore_index=True)
print(train_set)

                     0               1  2
0       UBERON:0004434  UBERON:0004313  1
1           HP:0033024      HP:0002720  1
2       UBERON:0015143  UBERON:0003916  1
3       UBERON:0010129  UBERON:0015052  1
4           HP:0033182      HP:0012379  1
...                ...             ... ..
122815      HP:0008935      HP:0011832  0
122816      GO:0043471      GO:0033044  0
122817  UBERON:0001479      GO:1904269  0
122818  UBERON:7500046      HP:0007755  0
122819  UBERON:0003387      HP:0025021  0

[122820 rows x 3 columns]


In [12]:
test_neg_x = []
test_neg_y = []

for i in range(len(test_neg_u)):
    position_x = val_list.index(test_neg_u[i])
    test_neg_x.append(key_list[position_x])
    position_y = val_list.index(test_neg_v[i])
    test_neg_y.append(key_list[position_y])

zero = [0]*len(test_neg_u)
test_neg = pd.DataFrame(list(zip(test_neg_x, test_neg_y, zero)))

In [13]:
test_pos_x = []
test_pos_y = []

test_pos_u = list(test_pos_u)
test_pos_v = list(test_pos_v)

for i in range(len(test_pos_u)):
    position_x = val_list.index(test_pos_u[i])
    test_pos_x.append(key_list[position_x])
    position_y = val_list.index(test_pos_v[i])
    test_pos_y.append(key_list[position_y])
  
ones = [1]*len(test_pos_u)
test_pos = pd.DataFrame(list(zip(test_pos_x, test_pos_y, ones)))

In [14]:
test_set = pd.concat([test_pos, test_neg], ignore_index=True)
print(test_set)

                 0           1  2
0       GO:1902868  GO:0065007  1
1       HP:0000108  HP:0100957  1
2       HP:0031783  HP:0011642  1
3       GO:1903797  GO:0065007  1
4       HP:0010197  HP:0010176  1
...            ...         ... ..
13641  CHEBI:18367  HP:0031255  0
13642   HP:0005363  HP:0031910  0
13643   HP:0003541  GO:0042488  0
13644    PR:Q62226  HP:0034097  0
13645   HP:0010715  HP:0100485  0

[13646 rows x 3 columns]


In [15]:
train_set.to_csv('/home/ukjung18/HiG2Vec/evalGO/HPOfull_samples_train.txt', header=False, index=False)

In [16]:
test_set.to_csv('/home/ukjung18/HiG2Vec/evalGO/HPOfull_samples.txt', header=False, index=False)

In [17]:
rel_type = ['subclass_of']*len(train_pos_x)
train_hpo = pd.DataFrame(list(zip(train_pos_x, train_pos_y, rel_type)))

In [18]:
train_hpo.to_csv("/home/ukjung18/HiG2Vec/data/HPOfull_train.tsv", mode='w', sep='\t', header=False, index=False)